•	“PaxDemand” sheet shows the average weekly demand of each “Zip2”.

•	“PaxDemandDist” sheet shows the average weekly demand’s breakdown by destination airport and class. For example, the data corresponds to Origin Zip2 “1”, Destination Airport “ABE” and “First Class” indicates a percentage of 0.074%, which means the weekly demand from “1” to “ABE” using “First Class” accounts for 0.074% of total demand out of “1”.

•	“TicketPrices” sheet shows the cost per ticket, given the Origin (i.e., Zip2), a Destination (i.e., an airport) and the class (i.e., First Class, Business Class, Main Cabin…).

•	“AirportRegions” sheet shows the region of each airport.

•	“Zip2Regions” sheet shows the region of each Zip2 location.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
from IPython.display import display, HTML
from statistics import mean
from statistics import pstdev

1.	(5 points) Determine the number of airports in each region.

In [2]:
AR = pd.read_csv('AirportRegions.csv')

Regions = AR.loc[:,'Region'].value_counts(dropna=False)

print("Region      #Airports")
display(Regions)

Region      #Airports


Northeast    219
Western      218
Midwest      151
Southeast    141
Central       90
Northwest     48
Southwest     26
US-Non-48      1
Mexico         1
Name: Region, dtype: int64

2.	(5 points) Which region contains the most Zip2 values?

In [3]:
Z2R = pd.read_csv('Zip2Regions.csv')

Zip2Count = Z2R['Region'].value_counts(dropna=False)

print("Region      #Zip2's")
Zip2Count.head(1)

Region      #Zip2's


Northeast    26
Name: Region, dtype: int64

3.	(10 points) Generate a new .csv file report that shows the expected weekly passenger arrivals and expected weekly revenue for each airport.

In [4]:
PD = pd.read_csv('PaxDemand.csv')
PDD = pd.read_csv('PaxDemandDist.csv')
TP = pd.read_csv('TicketPrices.csv')

# display(PD)
# display(PDD)
# display(TP)

PD.rename(columns={'Zip2': 'OrigZip2'}, inplace = True)
PDD.rename(columns={'FirstClass' : 'FirstClassPD',
                    'BusinessClass' : 'BusinessClassPD',
                    'MainCabin' : 'MainCabinPD',
                    'Economy' : 'EconomyPD'
                    }, inplace = True)

# print(PD.columns)
# print(PDD.columns)
# print(TP.columns)

#MERGE ALL DATA INTO ONE DF TO CALCULATE NEEDED ROWS
PDD1 = PDD.merge(PD, on='OrigZip2', how='left')
PDD2 = PDD1.merge(TP, left_on=['OrigZip2', 'DestAirport'], right_on=['OrigZip2', 'DestAirport'],how='left')



#CALCULATE EXPECTED WEEKLY PASSENGER ARRIVALS AND WEEKLY REVENUE
#Calc weekly passenger arrivals
PDD2['WklyArrivals'] = ((PDD2['FirstClassPD'] + PDD2['BusinessClassPD'] + PDD2['MainCabinPD'] + PDD2['EconomyPD'])* .01 * PDD2['WeeklyTicketFC']).round(0)
#Calc revenue for each ticket type
PDD2['FC_Revenue'] = PDD2['FirstClassPD'] * PDD2['WeeklyTicketFC'] * PDD2['FirstClass']
PDD2['BC_Revenue'] = PDD2['BusinessClassPD'] * PDD2['WeeklyTicketFC'] * PDD2['BusinessClass']
PDD2['MC_Revenue'] = PDD2['MainCabinPD'] * PDD2['WeeklyTicketFC'] * PDD2['MainCabin']
PDD2['EC_Revenue'] = PDD2['EconomyPD'] * PDD2['WeeklyTicketFC'] * PDD2['Economy']
PDD2['Sum_Revenue'] = (PDD2['FC_Revenue'] + PDD2['BC_Revenue'] + PDD2['MC_Revenue'] + PDD2['EC_Revenue']).round(2)

print(PDD2.columns)
# display(PDD2)

#CALCULATE SUMS FOR EACH AIRPORT
#use groupby each dest airport and sum target columns
SumByPort = PDD2.groupby(['DestAirport']).agg({'WklyArrivals': np.sum,
                                               'Sum_Revenue': np.sum})

display(SumByPort)
SumByPort.to_csv('AirportSummary.csv')

Index(['OrigZip2', 'DestAirport', 'FirstClassPD', 'BusinessClassPD',
       'MainCabinPD', 'EconomyPD', 'WeeklyTicketFC', 'Date', 'FirstClass',
       'BusinessClass', 'MainCabin', 'Economy', 'WklyArrivals', 'FC_Revenue',
       'BC_Revenue', 'MC_Revenue', 'EC_Revenue', 'Sum_Revenue'],
      dtype='object')


,WklyArrivals,Sum_Revenue
DestAirport,,
ABE,858.0,32512172.28
BFI,730.0,31750396.38
BNA,797.0,28626168.59
BWI,749.0,27641691.21
CAE,839.0,30747708.94
CHA,921.0,33875252.44
DFW,917.0,35973064.36
EWR,929.0,33419793.54
IND,852.0,30210098.31


4.	(10 points) Generate a new .csv file report that shows the expected weekly passenger arrivals and expected weekly revenue for each region.

In [5]:
PDD3 = PDD2.merge(AR, left_on=['DestAirport'], right_on=['Airport'], how='left')
SumByRegion = PDD3.groupby(['Region']).agg({'WklyArrivals': np.sum,
                                            'Sum_Revenue': np.sum})

SumByRegion.to_csv('RegionSummary.csv')

5.	(20 points) Remove a randomly generated airport as a possible destination for flights from a user-entered Zip2 (e.g., we can no longer fly from Zip2 “17” to airport “ABE”). Renormalize the percentages associated with each of the remaining airports and fare classes accordingly. To be clear, PaxDemand is not impacted by this change, but rather only the distribution is to be adjusted.

In [6]:
#create random number based on total number of airports
import random as rd
unlucky = rd.randrange(len(SumByPort.index)-1)

#filter out "unlucky" airport
Airport = SumByPort.index[unlucky]
AirportFilter = PDD['DestAirport'] != Airport
#print(Airport)

#store new PDD in new df
PDDnew = PDD[AirportFilter]

#Add line to store total demand aka pd + pd + pd + pd 
PDDnew['TotalPD'] = PDDnew['FirstClassPD'] + PDDnew['BusinessClassPD'] + PDDnew['MainCabinPD'] + PDDnew['EconomyPD']

#RENORMALIZE ALL PDs IN PDDnew
#group by zips and add demand
NewPDs = PDDnew.groupby(['OrigZip2']).agg({'TotalPD': np.sum})

NewPDs.rename(columns={'TotalPD': 'TotalZipPD'}, inplace=True)

#display(PDDnew)
#display(NewPDs)

#Make new df to merge PDDnew and NewPDs
PDDnew1 = PDDnew.merge(NewPDs, left_on=['OrigZip2'], right_on=['OrigZip2'],how='left')
#display(PDDnew1)

#Normalize all the PDs
PDDnew1['FirstClassPD'] = PDDnew1['FirstClassPD']/PDDnew1['TotalZipPD']*100
PDDnew1['BusinessClassPD'] = PDDnew1['BusinessClassPD']/PDDnew1['TotalZipPD']*100
PDDnew1['MainCabinPD'] = PDDnew1['MainCabinPD']/PDDnew1['TotalZipPD']*100
PDDnew1['EconomyPD'] = PDDnew1['EconomyPD']/PDDnew1['TotalZipPD']*100
PDDnew1['TotalPD'] = PDDnew1['FirstClassPD']+PDDnew1['BusinessClassPD']+PDDnew1['MainCabinPD']+PDDnew1['EconomyPD']

#Create a group and agg all demand to verify it adds to 100
#check1 = PDDnew1.groupby(['OrigZip2']).agg({'TotalPD': np.sum})
#display(check1)

#Create new PDD with adjusted values
PDDadjusted = PDDnew1[['OrigZip2','DestAirport','FirstClassPD','BusinessClassPD','MainCabinPD','EconomyPD']]
display(PDDadjusted)

C:\Users\James Sloan\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,OrigZip2,DestAirport,FirstClassPD,BusinessClassPD,MainCabinPD,EconomyPD
0,1,ABE,0.081502,3.398866,2.470400,0.421829
1,1,BFI,0.193843,4.742552,4.433063,0.506636
2,1,BNA,0.001101,1.646566,1.057327,0.083705
3,1,BWI,0.001101,0.832645,1.002258,0.098023
4,1,CAE,0.031940,0.528663,0.329313,0.145382
...,...,...,...,...,...,...
1563,99,ONT,0.413939,0.720113,10.117837,0.791621
1564,99,PHL,0.001007,0.012086,0.208480,0.359553
1565,99,PHX,0.103737,0.151073,4.918924,0.074529
1566,99,RIC,0.216537,1.235774,6.631081,0.350488


6.	(30 points) Assume you have learned that actual Passenger Demand follows a uniform distribution within +/-10% of the mean value of historical PaxDemand. Using the Uniform distribution, generate 100 instances of Passenger Demand using the given mean values in PaxDemand. Find the new mean value and standard deviation, then calculate the 95% confidence interval for Weekly Revenue based on this updated Passenger Demand and display that information in an appropriately formatted .csv output file.

In [7]:
instance = range(1,100)

#Store Mean and StdDev for 100 trials on FC demand by Zip
PD['MeanFC'] = 0
PD['StdDevFC'] = 0
for index in PD.index:
    UniformInst = []
    FCtest = PD.loc[index, 'WeeklyTicketFC']
    for inst in instance:
        UniformInst.append(rd.random()*(FCtest*1.1-FCtest*0.9)+FCtest*0.9)
    PD.loc[index,'MeanFC'] = mean(UniformInst)
    PD.loc[index,'StdDevFC'] = pstdev(UniformInst)

#Join MeanFC and StdDevFC into PDD2 to recalc Revenues
PDD4 = PDD2.merge(PD, left_on=['OrigZip2'], right_on=['OrigZip2'],how='left')
#Only needed columns
PDD4 = PDD4[['OrigZip2','DestAirport','FirstClassPD',
             'BusinessClassPD','MainCabinPD','EconomyPD',
             'FirstClass','BusinessClass','MainCabin',
             'Economy','MeanFC','StdDevFC']]

# #Recalculate weekly revenue and include 95% CI
# 95% CI
z = 1.96
PDD4['FC_95CI_UL'] = (PDD4['MeanFC']+PDD4['StdDevFC']/10).round(2)
PDD4['FC_95CI_LL'] = (PDD4['MeanFC']-PDD4['StdDevFC']/10).round(2)

PDD4['FC_Revenue'] = PDD4['FirstClassPD'] * PDD4['MeanFC'] * PDD4['FirstClass']
PDD4['FC_Revenue_UL'] = PDD4['FirstClassPD'] * PDD4['FC_95CI_UL'] * PDD4['FirstClass']
PDD4['FC_Revenue_LL'] = PDD4['FirstClassPD'] * PDD4['FC_95CI_LL'] * PDD4['FirstClass']

PDD4['BC_Revenue'] = PDD4['BusinessClassPD'] * PDD4['MeanFC'] * PDD4['BusinessClass']
PDD4['BC_Revenue_UL'] = PDD4['BusinessClassPD'] * PDD4['FC_95CI_UL'] * PDD4['BusinessClass']
PDD4['BC_Revenue_LL'] = PDD4['BusinessClassPD'] * PDD4['FC_95CI_LL'] * PDD4['BusinessClass']

PDD4['MC_Revenue'] = PDD4['MainCabinPD'] * PDD4['MeanFC'] * PDD4['MainCabin']
PDD4['MC_Revenue_UL'] = PDD4['MainCabinPD'] * PDD4['FC_95CI_UL'] * PDD4['MainCabin']
PDD4['MC_Revenue_LL'] = PDD4['MainCabinPD'] * PDD4['FC_95CI_LL'] * PDD4['MainCabin']

PDD4['EC_Revenue'] = PDD4['EconomyPD'] * PDD4['MeanFC'] * PDD4['Economy']
PDD4['EC_Revenue_UL'] = PDD4['EconomyPD'] * PDD4['FC_95CI_UL'] * PDD4['Economy']
PDD4['EC_Revenue_LL'] = PDD4['EconomyPD'] * PDD4['FC_95CI_LL'] * PDD4['Economy']

PDD4['Sum_Revenue'] = (PDD4['FC_Revenue'] + PDD4['BC_Revenue'] + PDD4['MC_Revenue'] + PDD4['EC_Revenue']).round(2)
PDD4['Sum_Revenue_UL'] = (PDD4['FC_Revenue_UL'] + PDD4['BC_Revenue_UL'] + PDD4['MC_Revenue_UL'] + PDD4['EC_Revenue_UL']).round(2)
PDD4['Sum_Revenue_LL'] = (PDD4['FC_Revenue_LL'] + PDD4['BC_Revenue_LL'] + PDD4['MC_Revenue_LL'] + PDD4['EC_Revenue_LL']).round(2)

# group by each zip and show CI for FC and Revenue
SumByZipWithCIs = PDD4.groupby(['OrigZip2']).agg({'FC_95CI_LL':np.sum,
                                                'FC_95CI_UL':np.sum,
                                                'Sum_Revenue_LL':np.sum,
                                                'Sum_Revenue_UL':np.sum})
SumByZipWithCIs.rename(columns={'FC_95CI_LL':'Forcast_Demand_LL',
                                'FC_95CI_UL':'Forcast_Demand_UL',
                                'Sum_Revenue_LL':'Revenue_LL',
                                'Sum_Revenue_UL':'Revenue_UL'}, inplace=True)
display(SumByZipWithCIs)
SumByZipWithCIs.to_csv('SummaryByZipWithCIs.csv')

,Forcast_Demand_LL,Forcast_Demand_UL,Revenue_LL,Revenue_UL
OrigZip2,,,,
1,1618.40,1636.93,3824212.26,3867997.90
2,635.80,642.77,1565839.96,1583005.60
3,631.21,638.35,1590056.23,1608042.33
4,3855.43,3902.01,7914655.31,8010277.51
5,1543.09,1561.11,3404159.39,3443912.70
...,...,...,...,...
95,3117.80,3154.18,7605094.57,7693834.47
96,1609.73,1627.75,5844409.21,5909834.03
97,13913.48,14071.24,33868680.01,34252704.92


7.	(10 points) Using PaxDemandDist, group all locations by OrigZip2 and find the mean value per class. The following is an example of how the grouped data should be appear:

Orig Zip2	Number of Airports	First Class	Business Class	Main Cabin	Economy

1	17	0.16	2.89	2.54	0.29

2	17	0.20	2.87	2.53	0.28

(…)	(…)	(…)	(…)	(…)	(…)

In [14]:
PDDsummary = PDD.groupby(['OrigZip2']).agg({'DestAirport':pd.Series.nunique,
                                             'FirstClassPD':np.mean,
                                             'BusinessClassPD':np.mean,
                                             'MainCabinPD':np.mean,
                                             'EconomyPD':np.mean})

PDDsummary.rename(columns={'DestAirport':'Number of Airports',
                   'FirstClassPD':'First Class',
                   'BusinessClassPD':'Business Class',
                   'MainCabinPD':'Main Cabin',
                   'EconomyPD':'Economy'}, inplace=True)
display(PDDsummary)

,Number of Airports,First Class,Business Class,Main Cabin,Economy
OrigZip2,,,,,
1,17,0.157588,2.888941,2.543529,0.292471
2,17,0.200176,2.869059,2.530000,0.283176
3,17,0.061647,3.257882,2.269176,0.293647
4,17,0.174294,3.078294,2.373059,0.256882
5,17,0.131471,2.992824,2.533824,0.224235
...,...,...,...,...,...
95,17,0.207000,0.605765,4.605471,0.464176
96,17,0.311235,0.785353,4.382412,0.403529
97,17,0.333176,0.634353,4.576647,0.338471


8.	(10 points) Create a .csv file containing a summary by quarter of the mean value per fare class contained in TicketPrices.

In [43]:
TP['Quarter'] = 0
TP['Year'] = 0
for index in TP.index:
    #pull date
    date = TP.loc[index,'Date']
    #determine quarter
    if '/' in date[:2]:
        if int(date[:1]) <= 3.1:
            quarter = 1
        elif int(date[:1]) <= 6.1:
            quarter = 2
        elif int(date[:1]) <= 9.1:
            quarter = 3
    else:
        quarter = 4
    #set quarter value
    TP.loc[index,'Quarter'] = quarter
    #determine year
    year = int(date[-4:])
    #set year value
    TP.loc[index,'Year'] = year

#Group by year and quarter and get average values
TpQuarterSummary = TP.groupby(['Year','Quarter']).agg({'FirstClass':np.mean,
                                                       'BusinessClass':np.mean,
                                                       'MainCabin':np.mean,
                                                       'Economy':np.mean})
display(TpQuarterSummary)
TpQuarterSummary.to_csv('TicketPriceQuarterSummary.csv')

FirstClass  BusinessClass   MainCabin     Economy
Year Quarter                                                   
2018 1        713.205000     429.205000  357.145000  339.340000
     2        750.792746     452.476684  365.984456  344.000000
     3        723.800000     439.068182  361.213636  341.604545
     4        756.377049     453.125683  369.371585  347.721311
2019 1        730.345361     440.525773  362.860825  341.154639
     2        747.227053     445.584541  366.217391  344.985507
     3        709.479798     433.994949  360.095960  339.848485
     4        719.979592     435.397959  360.826531  340.877551
2020 1        774.960000     461.213333  372.186667  349.533333